In [1]:
import gym 
import random

D:\anacondadata\envs\RL\lib\site-packages\gym\core.py:26: UserWarning: WARN: Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


In [2]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

D:\anacondadata\envs\RL\lib\site-packages\gym\envs\registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
D:\anacondadata\envs\RL\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
D:\anacondadata\envs\RL\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviou

In [3]:
print(actions)
print(states)

2
4


In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

env.close()

D:\anacondadata\envs\RL\lib\site-packages\gym\core.py:50: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  "You are calling render method, "


Episode:1 Score:19.0
Episode:2 Score:17.0
Episode:3 Score:21.0
Episode:4 Score:31.0
Episode:5 Score:52.0
Episode:6 Score:26.0
Episode:7 Score:9.0
Episode:8 Score:31.0
Episode:9 Score:11.0
Episode:10 Score:12.0


In [5]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [7]:
# 创建网络模型
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.linear1 = nn.Linear(input_dim, 128)
        self.linear2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x


In [8]:
# 创建环境
env = gym.make('CartPole-v1',render_mode="human")
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

D:\anacondadata\envs\RL\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
D:\anacondadata\envs\RL\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [9]:
# 创建模型和优化器
model = DQN(input_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters())
model

DQN(
  (linear1): Linear(in_features=4, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=2, bias=True)
)

In [10]:
# 创建记忆
memory = deque(maxlen=10000)

In [11]:
# 训练模型
for episode in range(4500):
    state = env.reset()
    state = torch.from_numpy(np.array(state)).float().to(device)  # 移动初始状态到设备
    total_reward = 0  # 初始化总奖励
    for t in range(200):
        action = env.action_space.sample() if random.random() < 0.1 else model(state).argmax().item()
        next_state, reward, done, _ = env.step(action)
        next_state = torch.from_numpy(np.array(next_state)).float().to(device)  # 移动 next_state 到设备
        total_reward += reward  # 累加奖励
        memory.append((state.cpu().numpy(), action, reward, next_state.cpu().numpy(), done))  # 将 state 和 next_state 移动回 CPU 并转换为 NumPy 数组
        state = next_state

        if done:
            break
    if episode % 100 == 0: 
        print(f"Episode {episode}, Total reward: {total_reward}")  # 输出这一轮的总奖励

    if len(memory) > 100:
        batch = random.sample(memory, 100)
        state, action, reward, next_state, done = zip(*batch)
        state = torch.from_numpy(np.array(state)).float().to(device)  # 移动 state 到设备
        action = torch.tensor(action).to(device)  # 移动 action 到设备
        reward = torch.tensor(reward).to(device)  # 移动 reward 到设备
        next_state = torch.from_numpy(np.array(next_state)).float().to(device)  # 移动 next_state 到设备
        done = torch.tensor(done).to(device)  # 移动 done 到设备

        q_values = model(state)
        next_q_values = model(next_state)

        q_value = q_values.gather(1, action.unsqueeze(1)).squeeze()
        next_q_value = next_q_values.max(1)[0]
        expected_q_value = reward + 0.99 * next_q_value * torch.logical_not(done)

        loss = (q_value - expected_q_value.detach()).pow(2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Episode 0, Total reward: 10.0
Episode 100, Total reward: 8.0
Episode 200, Total reward: 10.0
Episode 300, Total reward: 10.0
Episode 400, Total reward: 12.0
Episode 500, Total reward: 15.0
Episode 600, Total reward: 11.0
Episode 700, Total reward: 20.0
Episode 800, Total reward: 28.0
Episode 900, Total reward: 20.0
Episode 1000, Total reward: 64.0
Episode 1100, Total reward: 32.0
Episode 1200, Total reward: 73.0
Episode 1300, Total reward: 57.0
Episode 1400, Total reward: 30.0
Episode 1500, Total reward: 40.0
Episode 1600, Total reward: 53.0
Episode 1700, Total reward: 40.0
Episode 1800, Total reward: 122.0
Episode 1900, Total reward: 38.0
Episode 2000, Total reward: 28.0
Episode 2100, Total reward: 67.0
Episode 2200, Total reward: 81.0
Episode 2300, Total reward: 106.0
Episode 2400, Total reward: 124.0
Episode 2500, Total reward: 199.0
Episode 2600, Total reward: 200.0
Episode 2700, Total reward: 200.0
Episode 2800, Total reward: 200.0
Episode 2900, Total reward: 16.0
Episode 3000, To

KeyboardInterrupt: 

In [12]:
# 保存模型
torch.save(model.state_dict(), 'model.pth')

In [13]:
# 加载模型
model = DQN(input_dim, output_dim)  # 替换为你的模型类
model.load_state_dict(torch.load('model.pth'))
model.eval()  # 设置为评估模式

# 测试模型
total_reward = 0
state = env.reset()
env.reset()
for t in range(200):
    # env.render()
    with torch.no_grad():  # 不需要计算梯度
        action = model(torch.from_numpy(state).float()).argmax().item()
    state, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        env.reset()

env.close()
print("Total reward:", total_reward)

Total reward: 200.0
